# Stationäre Wärmeleitungsgleichung im Raum

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt

### Geometrie

Die folgende Geometrie repräsentiert einen beheizten Raum mit Wänden und Fenstern:

In [ ]:
def MakeRoom2DGeometry():
    wallthickness = 0.1
    distance_heater = 0.05
    heaterwidth = 0.3
    heaterheight = 0.1
    windowwidth = 0.3
    wall = Rectangle(1+2*wallthickness, 1+2*wallthickness).Face().Move((-wallthickness,-wallthickness,0))
    room = Rectangle(1, 1).Face()
    wall -= room
    heater = Rectangle(heaterwidth,heaterheight).Face().Move((0.5-0.5*heaterwidth,distance_heater,0))
    room -= heater
    basewindow = Rectangle(windowwidth,wallthickness).Face().Move((0.5-0.5*windowwidth,-wallthickness,0))
    windows = sum([basewindow.Rotate( Axis((0.5,0.5,0),Z),angle) for angle in [0,90,180,270]])
    room -= windows
    # label domains
    wall.faces.name="wall"; wall.faces.col = (0.4,0.4,0.4)
    room.faces.name="room"
    heater.faces.name="heater"; heater.faces.col = (1,0,0)
    windows.faces.name="windows"; windows.faces.col = (0,0,1)
    # label exterior edges
    wall.edges.Min(Y).name="exterior_wall"
    wall.edges.Min(X).name="exterior_wall"
    wall.edges.Max(X).name="exterior_wall"
    wall.edges.Max(Y).name="exterior_wall"
    windows.edges.Min(Y).name="exterior_windows"
    windows.edges.Min(X).name="exterior_windows"
    windows.edges.Max(X).name="exterior_windows"
    windows.edges.Max(Y).name="exterior_windows"
    geo = Glue([wall,room,heater,windows])

    return geo
geo = MakeRoom2DGeometry()
Draw(geo)
mesh2d = Mesh(OCCGeometry(geo,dim=2).GenerateMesh(maxh=0.1))

In [ ]:
def MakeRoom3DGeometry():
    wallthickness = 0.1
    distance_heater = 0.05
    heaterwidth = 0.5
    heaterheight = 0.3
    heaterdepth = 0.1
    windowwidth = 0.3
    windowheight = 0.3
    wall = Box((0,0,0), (1+2*wallthickness,1+2*wallthickness,1+2*wallthickness)).Move((-wallthickness,-wallthickness,-wallthickness))
    room = Box((0,0,0), (1,1,1))
    wall -= room

    heater = Box((0.5-0.5*heaterwidth,distance_heater,distance_heater),(0.5+0.5*heaterwidth,distance_heater+heaterdepth,distance_heater+heaterheight))
    room -= heater

    basewindow = Box((0.5-0.5*windowwidth,-wallthickness,0.5-0.5*windowheight),(0.5+0.5*windowwidth,0,0.5+0.5*windowheight))
    windows = sum([basewindow.Rotate( Axis((0.5,0.5,0),Z),angle) for angle in [0,90,180,270]])
    wall -= windows

    # label domains
    wall.solids.name="wall"; wall.faces.col = (0.4,0.4,0.4)
    room.solids.name="room"
    heater.solids.name="heater"; heater.faces.col = (1,0,0)
    windows.solids.name="windows"; windows.faces.col = (0,0,1)    

    # label exterior edges
    for axis in [X,Y]:
        wall.faces.Min(axis).name="exterior_wall"
        wall.faces.Max(axis).name="exterior_wall"
        windows.faces.Min(axis).name="exterior_windows"
        windows.faces.Max(axis).name="exterior_windows"
    wall.faces.Min(Z).name="bottom"
    wall.faces.Max(Z).name="top"

    geo = Glue([wall,room,windows,heater])
    return geo
geo = MakeRoom3DGeometry()
Draw(geo,clipping = { "pnt" : (0,0,0.5), "vec" : (0,0,-1)})
mesh3d = Mesh(OCCGeometry(geo,dim=3).GenerateMesh(maxh=0.15))

In [ ]:
# heat transfer coefficients:
alpha_vals = {'exterior_wall': Parameter(0.1),  'exterior_windows': Parameter(0.1)}
# heat conductivity:
lamb_vals = {'wall': Parameter(0.0015), 'room': Parameter(0.4), 'windows': Parameter(0.02), 'heater': Parameter(1.0)}
# heat source
source_vals = {'wall': Parameter(0), 'room': Parameter(0), 'windows': Parameter(0), 'heater': Parameter(50)}
# exterior temperature
Tref = 5
#Draw(lamb,mesh)

### Lösung der Wärmeleitungsgleichung

In [ ]:
def SolveHeatEquation(mesh, alpha_vals, lamb_vals, source_vals, Tref):
    alpha = mesh.BoundaryCF(alpha_vals, default=0)
    lamb = mesh.MaterialCF(lamb_vals, default=1)
    source = mesh.MaterialCF(source_vals, default=0)

    V = H1(mesh,order=3)
    T, v = V.TnT()

    a = BilinearForm(V)
    a += lamb*grad(T)*grad(v)*dx
    a += alpha*T*v*ds(mesh.Boundaries("exterior_wall|exterior_windows"))
    f = LinearForm(V)   
    f += source*v*dx(mesh.Materials("heater"))
    f += alpha*Tref*v*ds(mesh.Boundaries("exterior_wall|exterior_windows"))

    a.Assemble()
    f.Assemble()

    gfu = GridFunction(V)
    gfu.vec.data = a.mat.Inverse(V.FreeDofs()) * f.vec

    room_vol = Integrate(CF(1)*dx(mesh.Materials("room")),mesh)
    avg_temp = Integrate(gfu*dx(mesh.Materials("room")),mesh)/room_vol
    var_temp = sqrt (Integrate ((gfu-avg_temp)*(gfu-avg_temp)*dx(mesh.Materials("room")), mesh) \
                / (avg_temp * room_vol))
    print("Average room temperature:", avg_temp)
    print("Variance of room temperature:", var_temp)

    if mesh.dim == 2:
        print("temperature")
        Draw(gfu,mesh,"temperature")
        #print("heat flux")
        #Draw(-lamb*grad(gfu),mesh,"heat flux")
    else:
        print("temperature")
        Draw(gfu,mesh,"temperature", clipping = { "pnt" : (0,0,0.5), "vec" : (0,0,-1)})
        #print("heat flux")
        #Draw(-lamb*grad(gfu),mesh,"heat flux", clipping = { "pnt" : (0,0,0.5), "vec" : (0,0,-1)})


## 2D Wärmeleitungsgleichung

In [ ]:
source_vals["heater"].Set(50)
lamb_vals["windows"].Set(0.02)
SolveHeatEquation(mesh2d, alpha_vals, lamb_vals, source_vals, Tref)

## 3D Wärmeleitungsgleichung

In [ ]:
source_vals["heater"].Set(75)
lamb_vals["windows"].Set(0.02)
SolveHeatEquation(mesh3d, alpha_vals, lamb_vals, source_vals, Tref)